In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('apartment_rent_data.csv')
df_sorted = df.sort_values(by='total', ascending=True)
df_sorted

,id,title,rent,deposit,total
18,19,۷۰متر۱خواب ترکمنستان وزوایی,0,0,0
13,14,۶۴متر با آسانسور ۱۰ساله شریعتی. پلیس. کلیم کاشانی,1000000,700000000,733333333
9,10,شریعتی ملک ۶۸متر,17000000,200000000,766666666
4,5,۷۳متر آپارتمان مبله شیک ومجلل,18000000,200000000,800000000
20,21,۶۸ متری فول تکواحدی (سهروردی بهار شیراز)بی مشابه,0,870000000,870000000
5,6,بهار شیراز خداپرست ۶۲متر,100000,870000000,873333333
0,1,۶۳متر فول امکانات شریعتی. سهروردی. ملک,8500000,600000000,883333333
11,12,۶۰متر سهروردی شمالی/فول امکانات,18000000,300000000,900000000
8,9,۶۵متر ده ساله فول امکانات بازسازی سیدخندان-دبستان,0,900000000,900000000
2,3,۶۵متر۱خواب فول امکانات ۱۵مترپاسیومسقف.شریعتی.ملک,9000000,600000000,900000000
